# 라이브러리 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install finance-datareader
!pip install ta

In [3]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from statsmodels.tsa.seasonal import seasonal_decompose

import ta
from xgboost import XGBRegressor

import os 
from tqdm import tqdm
import warnings 
warnings.filterwarnings(action = 'ignore') 
pd.options.display.max_columns = None

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 데이터셋 로드

In [4]:
path = '/content/drive/MyDrive/stock_2/'

stock_list = pd.read_csv(path + 'stock_list.csv')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6))
stock_codes = np.sort(stock_list['종목코드'].values)

In [5]:
# start_date = '20210104'
# end_date = '20211105'

# start_weekday = pd.to_datetime(start_date).weekday()
# max_weeknum = pd.to_datetime(end_date).strftime('%V')
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

# print(f'WEEKDAY of "start_date" : {start_weekday}')
# print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
# print(f'HOW MANY "Business_days" : {Business_days.shape}', )
# display(Business_days.head())

# 전처리를 위한 함수 정의

https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html

https://junyoru.tistory.com/136

In [6]:
# 이동평균 추가하는 함수 정의 
def add_rolling_mean(df:pd.DataFrame, col_name:str, window_list:list)->pd.DataFrame:
  for window in window_list:
    df[col_name + f'_rolling{window}'] = df[col_name].rolling(window).mean().fillna(method = 'bfill') 
  return df

# 컬럼값을 lag하는 함수 정의
def lag_features(df:pd.DataFrame, col_list:list, lag_num:int)->pd.DataFrame:
  for col in col_list:
    df[col] = df[col].shift(lag_num).fillna(method = 'ffill').fillna(method = 'bfill')
  return df
  

def stock_indicators(df):
    # trend indicators
    df['rsi'] = ta.momentum.rsi(df['Close'])
    df['macd'] = ta.trend.macd(df['Close'])
    # df['obv'] = ta.volume.on_balance_volume(df['Close'],df['Volume'])
    df['aroon_up'] = ta.trend.aroon_up(df['Close'],window=25)
    df['aroon_down'] = ta.trend.aroon_down(df['Close'],window=25)
    # df['dpo'] = ta.trend.dpo(df['Close'], window=20)
    # df['ema'] = ta.trend.ema_indicator(df['Close'], window=14)
    df['ichimoku_a'] = ta.trend.ichimoku_a(df['High'], df['Low'], window1=9, window2=26)
    df['ichimoku_b'] = ta.trend.ichimoku_b(df['High'], df['Low'], window2=26, window3=52)
    df['ichimoku_base_line'] = ta.trend.ichimoku_base_line(df['High'], df['Low'], window1=9, window2=26)
    # df['ichimoku_conversion_line'] = ta.trend.ichimoku_conversion_line(df['High'], df['Low'], window1=9, window2=26)
    # df['kst'] = ta.trend.kst(df['Close'], roc1=10, roc2=15, roc3=20, roc4=30, window1=10, window2=10, window3=10, window4=15)
    # df['mass'] = ta.trend.mass_index(df['High'], df['Low'], window_fast=9, window_slow=25)

    # volatility indicators
    df['bol_h'] = ta.volatility.bollinger_hband(df['Close'])
    df['bol_avg'] = ta.volatility.bollinger_mavg(df['Close'])
    df['bol_l'] = ta.volatility.bollinger_lband(df['Close'])    

    # momentum
    # df['ppo'] = ta.momentum.ppo(df['Close'])
    # df['pvo'] = ta.momentum.pvo(df['Close'])
    # df['roc'] = ta.momentum.roc(df['Close'])

    return df

# 종목별 전처리

In [7]:
stock_df_dict = {}

nas = fdr.DataReader('IXIC', start = '20110104', end = '20211128')
nas.reset_index(inplace=True, drop=False)
nas.drop(columns=['Volume'], inplace=True)
nas.columns = ['Date', 'nas_close', 'nas_open', 'nas_high', 'nas_low', 'nas_change']
nas = nas[['Date', 'nas_close']]
djs = fdr.DataReader('DJI', start = '20110104', end = '20211128')
djs.reset_index(inplace=True, drop=False)
djs.drop(columns=['Volume'], inplace=True)
djs.columns = ['Date', 'djs_close', 'djs_open', 'djs_high', 'djs_low', 'djs_change']
djs = djs[['Date', 'djs_close']]
usd = fdr.DataReader('USD/KRW', start = '20110104', end = '20211128')
usd.reset_index(inplace=True, drop=False)
usd.columns = ['Date', 'usd_close', 'usd_open', 'usd_high', 'usd_low', 'usd_change']
usd = usd[['Date', 'usd_close']]


# 종목별로 stock_df 만들어서 딕셔너리 stock_df_dict에 저장하기 
for stock_code in tqdm(stock_codes): 

  # 날짜 설정
  dates = pd.date_range('20110104', '20211203', freq = 'B')
  df = pd.DataFrame(dates, columns = ['Date'])
  
  # 연, 월, 주, 일, 요일, 월별주수 정보 추가
  df['Year'] = df['Date'].dt.year
  df['Month'] = df['Date'].dt.month
  df['Week'] = df['Date'].dt.week
  df['Day'] = df['Date'].dt.day
  df['Weekday'] = df['Date'].dt.weekday
  df['WeekNum'] = np.ceil(df['Day']/7).astype(int)


  # 20110101 ~ 20210910 까지의 주가 정보 가져오기
  stock_value = fdr.DataReader(stock_code, start = '20110104', end = '20211128').reset_index()
  df = pd.merge(df, stock_value, on = 'Date', how = 'left')
  df = pd.merge(df, nas, on = 'Date', how = 'left')
  df = pd.merge(df, djs, on = 'Date', how = 'left')
  df = pd.merge(df, usd, on = 'Date', how = 'left')
  df[df.columns] = df[df.columns].ffill() # 결측치를 이전 값으로 대체 (공휴일 등 쉬는 날에 대하여)
  df = stock_indicators(df)
  df = df.dropna() # 남아있는 결측치는 상장전이므로 drop 

  # 종가 데이터 시계열 분해
  df = df.set_index('Date') 
  decomposition = seasonal_decompose(df[:-5]['Close'], model='multiplicative') 
  df['Close_trend'] = decomposition.trend.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 추세 
  df['Close_seosonal'] = decomposition.seasonal.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 계절성 

  # 5일, 20일, 60일, 120일, 240일 이동평균 추가 
  df = add_rolling_mean(df, 'Close', [5, 20, 60, 120, 240]) 
  
  # lag features 
  col_list = list(df.iloc[:, 6:].columns)
  col_list.remove('Close')
  df = lag_features(df, col_list, 5)
  df.drop(df.index[:5])

  df = df.reset_index()
  df['Index'] = df.index 
  stock_df_dict[stock_code] = df

100%|██████████| 370/370 [11:27<00:00,  1.86s/it]


# 모델링 (XGBoost)

In [8]:
# SEED 설정 
SEED = 42

# NMAE 함수 정의
def NMAE(true, pred):
 return np.mean(abs(true - pred) / true) * 100 

## public

In [9]:
# public 
xgb_oof_score = []
xgb_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (public)
  train = stock_df_dict[stock_code][:-25]
  test = stock_df_dict[stock_code][-25:-20]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  xgb = XGBRegressor(n_estimators= 10000, learning_rate = 0.05, tree_method = 'gpu_hist', random_state = SEED)
  xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  xgb_oof = xgb.predict(X_valid)
  xgb_pred = xgb.predict(X_test)
  xgb_pred_dict[stock_code] = xgb_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, xgb_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  xgb_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Public OOF NMAE Score : {np.mean(xgb_oof_score)}')

  0%|          | 0/370 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
[13:36:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:27416.6
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:766.655
Stopping. Best iteration:
[69]	validation_0-mae:430.252



  0%|          | 1/370 [00:00<03:07,  1.96it/s]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.507339


[종목코드 000080 종목명 하이트진로 학습 및 예측]
[13:36:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:34217.7
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:984.085
[200]	validation_0-mae:832.838
[300]	validation_0-mae:821.923
[400]	validation_0-mae:847.513
Stopping. Best iteration:
[324]	validation_0-mae:818.438



  1%|          | 2/370 [00:01<03:43,  1.65it/s]

...


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.886798


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]
[13:39:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:26035.2
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:326.505
Stopping. Best iteration:
[81]	validation_0-mae:136.284



100%|█████████▉| 369/370 [02:54<00:00,  2.29it/s]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 0.498166


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]
[13:39:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:16603.6
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:1211.72
[200]	validation_0-mae:1084.58
[300]	validation_0-mae:1033.65
[400]	validation_0-mae:1005.03
Stopping. Best iteration:
[323]	validation_0-mae:979.651



100%|██████████| 370/370 [02:55<00:00,  2.11it/s]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 5.561883

전체 종목에 대한 평균 Public OOF NMAE Score : 2.482989840230546


In [10]:
 xgb_public_sub = pd.DataFrame({'Day' : pd.date_range('20211101', '20211105', freq = 'B')})

for stock_code, pred in xgb_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  xgb_public_sub = pd.concat([xgb_public_sub, stock_pred], axis = 1)

xgb_public_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29042.0,36580.507812,60798.699219,148116.734375,102711.203125,16756.503906,49344.781250,83316.226562,100292.351562,710238.4375,51795.023438,250553.109375,34197.269531,55687.050781,16927.513672,2235.137695,28015.287109,5302.240234,334124.06250,51977.250000,13159.511719,32881.062500,9632.071289,7674.044434,31184.888672,145877.12500,47341.527344,85158.085938,45274.441406,255075.390625,289237.75000,82975.320312,104841.398438,33893.671875,29314.751953,33192.707031,156864.109375,210509.000000,97992.945312,98960.007812,306159.7500,64891.894531,34299.550781,70175.859375,64461.816406,13246.646484,65443.042969,260595.734375,41919.562500,739078.5625,186454.156250,9064.643555,8897.161133,32773.050781,487372.28125,18399.097656,4842.699219,89417.046875,58467.050781,158038.750000,101600.828125,100437.335938,42559.675781,29559.232422,131372.562500,60141.042969,553847.3125,71693.078125,46279.347656,102995.734375,26467.619141,208532.437500,232462.781250,26773.425781,87172.539062,171992.71875,264601.843750,44321.128906,87947.835938,84722.695312,5404.214844,77473.476562,330909.34375,6121.655762,22894.345703,48773.308594,11258.820312,302770.187500,48124.214844,157033.765625,14844.44043,57390.808594,104770.804688,81224.507812,6776.062012,105388.593750,11240.898438,87007.023438,12770.919922,32030.792969,4548.278809,24998.548828,120495.242188,42333.128906,6152.511230,35294.492188,24886.710938,

## private

In [11]:
# private
xgb_oof_score = []
xgb_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (private)
  train = stock_df_dict[stock_code][:-5]
  test = stock_df_dict[stock_code][-5:]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  xgb = XGBRegressor(n_estimators= 10000, learning_rate = 0.05, tree_method = 'gpu_hist', random_state = SEED)
  xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  xgb_oof = xgb.predict(X_valid)
  xgb_pred = xgb.predict(X_test)
  xgb_pred_dict[stock_code] = xgb_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, xgb_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Private OOF NMAE Score : {nmae:.6f}\n")
  xgb_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Private OOF NMAE Score : {np.mean(xgb_oof_score)}')

  0%|          | 0/370 [00:00<?, ?it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
[13:39:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:29097.8
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:1523.93
[200]	validation_0-mae:1357.61
Stopping. Best iteration:
[165]	validation_0-mae:1346.09



  0%|          | 1/370 [00:00<02:42,  2.28it/s]


2021-11-22 ~ 2021-11-26 기간동안의 Private OOF NMAE Score : 4.314211


[종목코드 000080 종목명 하이트진로 학습 및 예측]
[13:39:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:30575.9
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:1282.02
Stopping. Best iteration:
[64]	validation_0-mae:567.249



  1%|          | 2/370 [00:00<02:09,  2.84it/s]

...


2021-11-22 ~ 2021-11-26 기간동안의 Private OOF NMAE Score : 11.618504


[종목코드 363280 종목명 티와이홀딩스 학습 및 예측]
[13:42:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:23674
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:319.499
Stopping. Best iteration:
[75]	validation_0-mae:150.931



100%|█████████▉| 369/370 [02:38<00:00,  3.16it/s]


2021-11-22 ~ 2021-11-26 기간동안의 Private OOF NMAE Score : 0.596499


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]
[13:42:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:14866.1
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:854.463
Stopping. Best iteration:
[50]	validation_0-mae:484.041



100%|██████████| 370/370 [02:38<00:00,  2.34it/s]


2021-11-22 ~ 2021-11-26 기간동안의 Private OOF NMAE Score : 2.979636

전체 종목에 대한 평균 Private OOF NMAE Score : 2.48931903128594


In [12]:
xgb_private_sub = pd.DataFrame({'Day' : pd.date_range('20211129', '20211203', freq = 'B')})

for stock_code, pred in xgb_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  xgb_private_sub = pd.concat([xgb_private_sub, stock_pred], axis = 1)

xgb_private_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-29,29338.689453,31560.478516,59670.304688,134095.71875,125664.484375,15572.558594,44962.601562,83111.812500,113821.695312,638997.7500,45845.746094,207421.343750,32383.816406,67061.757812,15529.832031,1897.262939,25159.654297,5091.312012,303922.00000,45103.929688,12907.502930,30288.439453,9910.452148,7753.062012,28158.291016,143884.828125,42299.433594,83245.289062,40635.515625,225004.578125,293140.90625,82759.562500,97294.046875,31640.160156,26002.242188,34281.207031,139782.375000,213099.421875,98056.203125,101146.164062,277920.96875,54849.136719,32931.042969,72940.640625,65279.816406,12975.602539,54381.476562,226338.421875,38530.226562,727290.9375,175129.781250,9016.868164,9075.540039,31703.494141,460786.90625,17696.343750,5025.034668,77091.132812,53563.500000,172511.062500,87955.234375,96357.859375,35588.121094,27606.796875,114692.992188,56697.593750,506003.15625,74464.531250,43271.531250,84565.140625,25901.353516,260121.343750,213263.406250,24449.742188,78970.531250,156502.859375,246530.31250,43372.847656,77491.195312,75823.632812,5090.090820,85396.101562,320034.09375,6027.567383,21803.835938,46540.347656,11171.489258,309490.90625,43087.195312,152471.000000,13609.701172,36150.078125,119617.945312,71922.945312,6422.355469,91901.976562,11025.965820,105460.289062,9617.082031,35461.476562,4204.933105,21984.632812,113502.281250,38611.312500,5445.163086,33813.078125,23635.9003

# Submission

In [13]:
xgb_sub = pd.concat([xgb_public_sub, xgb_private_sub], ignore_index = True)
xgb_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29042.000000,36580.507812,60798.699219,148116.734375,102711.203125,16756.503906,49344.781250,83316.226562,100292.351562,710238.4375,51795.023438,250553.109375,34197.269531,55687.050781,16927.513672,2235.137695,28015.287109,5302.240234,334124.06250,51977.250000,13159.511719,32881.062500,9632.071289,7674.044434,31184.888672,145877.125000,47341.527344,85158.085938,45274.441406,255075.390625,289237.75000,82975.320312,104841.398438,33893.671875,29314.751953,33192.707031,156864.109375,210509.000000,97992.945312,98960.007812,306159.75000,64891.894531,34299.550781,70175.859375,64461.816406,13246.646484,65443.042969,260595.734375,41919.562500,739078.5625,186454.156250,9064.643555,8897.161133,32773.050781,487372.28125,18399.097656,4842.699219,89417.046875,58467.050781,158038.750000,101600.828125,100437.335938,42559.675781,29559.232422,131372.562500,60141.042969,553847.31250,71693.078125,46279.347656,102995.734375,26467.619141,208532.437500,232462.781250,26773.425781,87172.539062,171992.718750,264601.843750,44321.128906,87947.835938,84722.695312,5404.214844,77473.476562,330909.34375,6121.655762,22894.345703,48773.308594,11258.820312,302770.187500,48124.214844,157033.765625,14844.440430,57390.808594,104770.804688,81224.507812,6776.062012,105388.593750,11240.898438,87007.023438,12770.919922,32030.792969,4548.278809,24998.548828,120495.242188,42333.128906,6152.511230,35294.492188,248

In [14]:
xgb_sub.to_csv(path + 'xgboost_feature.csv', index = False)